In [5]:
%pip install --upgrade pip setuptools wheel
%pip install -q -U -r notebook-requirements.txt

Note: you may need to restart the kernel to use updated packages.
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [33 lines of output]
      Traceback (most recent call last):
        File "/home/daines/miniconda3/envs/archon/lib/python3.12/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 353, in <module>
          main()
        File "/home/daines/miniconda3/envs/archon/lib/python3.12/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "/home/daines/miniconda3/envs/archon/lib/python3.12/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 112, in get_requires_for_build_wheel
          backend = _build_backend()
                    ^^^^^^^^^^^^^^^^
        File "/home/daines/minico

In [1]:
from attrs import define, field
from batchfold.batchfold_job import BatchFoldJob
from datetime import datetime
import logging
import os

In [2]:
@define(kw_only=True)
class SuperfoldJob(BatchFoldJob):
    """Superfold Job"""

    input_s3_uri: str
    output_s3_uri: str
    params: dict = {}
    weights_dir: str = "/database"
    output_dir: str = "outputs/"
    job_name: str = field(default="SuperfoldJob" + datetime.now().strftime("%Y%m%d%s"))
    job_definition_name: str = field(default="SuperfoldJobDefinition")
    cpu: int = field(default=4)
    memory: int = field(default=16)
    gpu: int = field(default=1)

    def __attrs_post_init__(self) -> None:
        """Override default BatchFoldJob command"""
        input_filename = os.path.basename(self.input_s3_uri)

        command_list = [f"-i {self.input_s3_uri}:/opt/superfold/inputs/"]
        command_list.extend([f"-o outputs/:{self.output_s3_uri}"])
        command_list.extend(
            [
                "python /opt/superfold/run_superfold.py",
                f"/opt/superfold/inputs/{input_filename}",
                f"--alphafold_weights {self.weights_dir}",
                f"--out_dir {self.output_dir}",
            ]
        )
        command_list.extend([f"{key}=\'{value}\'" for key, value in self.params.items()])
        logging.info(f"Command is \n{command_list}")
        self.define_container_overrides(command_list, self.cpu, self.memory, self.gpu)
        return None


In [4]:
import boto3
from batchfold.batchfold_environment import BatchFoldEnvironment
from batchfold.utils import utils
from datetime import datetime
import os
import json
import time
from math import ceil

# boto3.session.Session(profile_name='myprofile'): 
# This creates a session using the credentials and configuration 
# stored under the [myprofile] profile in your 
# ~/.aws/credentials and ~/.aws/config files.

boto_session = boto3.session.Session(profile_name='archon')
s3 = boto_session.client("s3")
batch_environment = BatchFoldEnvironment(boto_session=boto_session)

S3_BUCKET = batch_environment.default_bucket
#S3_PREFIX = "superfold-experiment-" + datetime.now().strftime("%Y%m%d%s")
S3_PREFIX = "superfold-experiment-202406071717727382"
S3_PATH = os.path.join("s3://", S3_BUCKET, S3_PREFIX)
JOB_QUEUE = "G4dnSpotJobQueue"
print(f"S3 path is {S3_PATH}")
input_filename = "test.fa"

S3 path is s3://archon-iac-nebula-batchfolds3bucket-lxwt2piq2iwo/superfold-experiment-202406071717727382


In [5]:
MODELS = 4
MAX_RECYCLES = 5

params = {
    "--models": MODELS,
    "--max_recycles": MAX_RECYCLES
}


superfold_job_name = "SuperfoldJob" + datetime.now().strftime("%Y%m%d%s")

superfold_job = SuperfoldJob(
    boto_session=boto_session,
    job_name=superfold_job_name,
    input_s3_uri= S3_PATH + "/" + input_filename,
    output_s3_uri=S3_PATH + "/" + superfold_job_name,
    params=params,
)
print(f"Submitting {superfold_job_name}")
superfold_submission = batch_environment.submit_job(superfold_job, job_queue_name=JOB_QUEUE)


Submitting SuperfoldJob202406111718129597
